# Libraries

In [1]:
# Data 
import pandas as pd
import numpy as np
import pyarrow
import pandas as pd
import missingno as msno
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import warnings

# Modelling
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegressionCV
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import  StandardScaler
from sklearn.metrics import average_precision_score, classification_report,  roc_auc_score, accuracy_score, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV

## SHAP plot
import shap
import matplotlib.pyplot as plt

# Admin
import pickle
import gc


# Data

In [113]:
data_all = pd.read_parquet('../MIMICED/ed_join_cc.parquet')
cc = pd.read_parquet('../MIMICED/chiefcomp_clustered.parquet')
cc = cc.drop(columns=['chiefcomplaint','cc_cleaned'])

Join the files by stay_id

In [114]:
data_all = data_all.merge(cc, how='left', on='stay_id')

In [ ]:
data_all.info()

## Wrangle

Select columns for model

In [116]:
cols_keep = [ 
    'gender', 
    'age',
    'race',
    'arrival_transport',
    'pain',
    'temperature',
    'heartrate',
    'resprate',
    'o2sat',
    'sbp',
    'dbp',
    'subject_id',
    'intime',
    'icd_chapter',
    'revisit_72hrs',
    'med_gsn_count',
    'prior_visits_1yr',
    'acuity',
    'cci_score', 
    'cluster', 
    'res',
    'nyu', 
    'disposition'
    ]
data = data_all[cols_keep].copy()

Make the CCI into as category with fewer levels

In [117]:
data['CCI_cat'] = np.where(data['cci_score'] >= 5, '6', data['cci_score'].astype(str))
data = data.drop(columns=['cci_score'])

Recode the number of visits in the last 12 months into a category

In [118]:
data['previous_cat'] = np.where(data['prior_visits_1yr'] >= 1, 1, 0)
data = data.drop(columns=['prior_visits_1yr'])

Get the previous diagnosis (as icd chapter) if there was one

In [119]:
data = data.sort_values(by=['subject_id', 'intime'])
data['icd_chapter_lag'] = data.groupby('subject_id')['icd_chapter'].shift(1)
data['previous_result'] = np.where(
    data['previous_cat'] == 1,
    data['icd_chapter_lag'],
    'none'
)
data = data.drop(columns=['icd_chapter_lag', 'icd_chapter'])

Create a time category

In [120]:
def categorize_time(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

data['time_of_day'] = data['intime'].dt.hour.apply(categorize_time)


Create the outcome admitted 

In [121]:
data['home'] = data['disposition'].apply(lambda x: 1 if x == 'HOME' else 0)
data = data.drop(columns=['disposition'])

Ensure categpries are categories

In [122]:
data['nyu'] = data['nyu'].astype('int64')
data['res'] = data['res'].astype('int64')

columns_cat = ['gender', 'race','arrival_transport', 'revisit_72hrs','CCI_cat','previous_cat', 'cluster','time_of_day', 'nyu', 'res' , 'home', 'previous_result']    
for col in columns_cat:      
    data[col] = data[col].astype('category')

For meds count, NA = 0

In [123]:
data['med_gsn_count'] = data['med_gsn_count'].fillna(0)

Group the race variable

In [124]:
conditions = [
    data['race'].str.contains('WHITE', case=False, na=False),
    data['race'].str.contains('BLACK', case=False, na=False),
    data['race'].str.contains('HISPANIC', case=False, na=False),
    data['race'].str.contains('ASIAN', case=False, na=False)
]
choices = ['White', 'Black', 'Hispanic/Latino', 'Asian']
data['race'] = np.select(conditions, choices, default='Other')

Keep only the last visit (i.e. one per subject, but preserving info about previous visits)

In [125]:
last_visit_idx = data.groupby('subject_id')['intime'].idxmax()
data = data.loc[last_visit_idx].reset_index(drop=True)
data = data.drop(columns=['intime', 'subject_id'])

## Relationships

In [126]:
# Function to plot variable distributions
def plot_function(df, outcome):
    # Suppress warnings about ticks
    warnings.filterwarnings('ignore')
    # var types
    cont_vars  = df.drop(columns = [outcome]).select_dtypes(include=['int','float']).columns.tolist()
    cat_vars = df.drop(columns =[outcome]).select_dtypes(include=['category', 'object']).columns.tolist()

    # plot dim
    num_plots = len(cont_vars) + len(cat_vars) 
    cols = 3
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4))
    axes = axes.flatten()
    plot_idx = 0

    # plot continuous
    for col in cont_vars:
        sns.kdeplot(data=df, x=col, hue=outcome, fill=True, common_norm=False, ax=axes[plot_idx])
        axes[plot_idx].set_title(f'Distribution of {col} by {outcome}')
        plot_idx += 1

    # plot cat    
    for col in cat_vars:
        sns.countplot(data=df, x=col, hue=outcome, ax=axes[plot_idx])
        axes[plot_idx].set_title(f'{col} counts by {outcome}')
        axes[plot_idx].set_xticklabels([label.get_text()[:3] for label in axes[plot_idx].get_xticklabels()], rotation=45)
        plot_idx += 1

    # Remove unused axes
    for i in range(plot_idx, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot correlations
import seaborn as sns
sns.pairplot(data.drop(columns=['nyu','res', 'home']), corner=True)

#### Home

In [ ]:
plot_function(data.drop(columns=['nyu','res']), 'home')

### Diagnosis

In [ ]:
plot_function(data.drop(columns=['home','res']), 'nyu')

### Resources

In [ ]:
plot_function(data.drop(columns=['home','nyu']), 'res')

## Missing

Print missing data total

In [ ]:
print(data.drop(columns=['nyu', 'res','home']).isna().sum())

In [ ]:
msno.bar(data.drop(columns=['nyu', 'res', 'home']))

In [ ]:
msno.heatmap(data.drop(columns=['nyu', 'res', 'home']))

Create an imputed dataset using median impute

In [134]:
m_cols = ['pain', 'age', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'acuity']
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
data_impute = data.copy()
data_impute[m_cols] = imputer.fit_transform(data[m_cols])

In [ ]:
msno.bar(data_impute.drop(columns=['nyu', 'res',  'home']))

# Prediction

## Functions

In [146]:
# Format data
def data_process(df_in, predictors, target_columns, drop_first_in=False, tab = 'not_tab'):

   # Select relevant columns
    data_predict = df_in[predictors + target_columns].copy()
    
    # Identify column types
    numerical_cols = data_predict[predictors].select_dtypes(include='number').columns.tolist()
    categorical_cols = data_predict[predictors].select_dtypes(include=['object', 'category']).columns.tolist()

    if tab == 'tab':
        cat_idxs = []
        cat_dims = []
        for i, col in enumerate(categorical_cols):
            le = LabelEncoder()
            data_predict[col] = le.fit_transform(data_predict[col])
            cat_idxs.append(data_predict.columns.get_loc(col))
            cat_dims.append(data_predict[col].nunique())

    # Train-test split
    train_df, test_df = train_test_split(
        data_predict, 
        test_size=0.2, 
        random_state=42, 
        stratify=data_predict[target_columns]
    )
    
    # Separate X and y
    X_train = train_df[predictors].copy()
    X_test = test_df[predictors].copy()
    y_train = train_df[target_columns].copy()
    y_test = test_df[target_columns].copy()

    if tab == 'not_tab':
        # One-hot encode categorical columns
        X_train = pd.get_dummies(
        X_train,
        columns=categorical_cols,
        drop_first=drop_first_in
        )
        X_test = pd.get_dummies(
        X_test,
        columns=categorical_cols,
        drop_first=drop_first_in
        )

    # Align columns
    X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

    # Sanity checks
    print('Train X shape:', X_train.shape)
    print("Test X shape:", X_test.shape)

    print(f"\nCheck proportions of outcome in data:")
    for col in y_train.columns:
        print(f"--- {col} ---")
        print("y_train proportions:")
        print(y_train[col].value_counts(normalize=True).sort_index())
        print("y_test proportions:")
        print(y_test[col].value_counts(normalize=True).sort_index())

    # Return fully separated objects
    return X_train, X_test, y_train, y_test

In [147]:
# XGBoost 
def xgb_f(X_train, y_train, X_test, y_test,rename_map, outcome, shap_in = True):
    # Get original names for SHAP
    original_names = X_train.columns.tolist()

    # Calculate scale_pos_weight
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

    # Model
    clf = XGBClassifier(
        objective='binary:logistic',
        eval_metric='aucpr',
        random_state=42,
        scale_pos_weight=scale_pos_weight[outcome]
    )
    # Fit
    model = clf.fit(X_train, y_train[outcome])
    calibrator = CalibratedClassifierCV(model, method='isotonic', cv=5)
    calibrator.fit(X_train, y_train[outcome].astype(int))
    #Predict 
    y_pred = calibrator.predict(X_test)
    y_proba = calibrator.predict_proba(X_test)[:, 1]
    # Calculate AUC-PR with CI
    n_bootstraps = 1000
    rng = np.random.RandomState(42)  
    bootstrapped_scores = []
    y_test_array = np.array(y_test)
    y_proba_array = np.array(y_proba)

    for _ in range(n_bootstraps):
        # Sample with replacement
        indices = rng.choice(range(len(y_proba_array)), size=len(y_proba_array), replace=True)
        score = average_precision_score(y_test_array[indices], y_proba_array[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.sort(bootstrapped_scores)
    ci_lower = np.percentile(sorted_scores, 2.5)
    ci_upper = np.percentile(sorted_scores, 97.5)
    print(f"PR-AUC: {average_precision_score(y_test_array, y_proba_array):.2f}")
    print(f"95% CI: ({ci_lower:.2f}, {ci_upper:.2f})")
    # Calculate AUC
    auc = roc_auc_score(y_test, y_proba)
    print(f'AUC: {auc:.2f}')
    # Print report
    print(f"{classification_report(y_test, y_pred, digits=2)}\n")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")

    # 4. Compute Spiegelhalter Z-statistic
    E = np.sum(y_proba)  
    O = np.sum(y_test.astype(int).values)  
    V = np.sum(y_proba * (1 - y_proba))  
    Z = (O - E) / np.sqrt(V)
    print(f"Spiegelhalter Z-statistic: {Z:.3f}\n")

    # SHAP
    if shap_in == False:
        return
    else:    
        X_train_df = pd.DataFrame(X_train, columns=original_names)
        X_train_renamed = X_train_df.rename(columns=rename_map)
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_train_renamed)

        ## Table
        feature_importance = np.abs(shap_values).mean(axis=0)
        importance_df = pd.Series(feature_importance, index=X_train_renamed.columns)
        top_10_features = importance_df.sort_values(ascending=False).head(10)
        print("Top 10 most important features by mean |SHAP| value:")
        print(top_10_features)
    
        ## Plot
        shap.summary_plot(
            shap_values, 
            X_train_renamed, 
            max_display=5,
            show=False
        )
        fig = plt.gcf()
        ax = plt.gca()
        fig.text(
            x=0.01,             
            y=0.95,             
            s=outcome,           
            fontsize=14,
            color='black',
            ha='left',         
            va='top'            
        )
        # Set font color to black
        for text in ax.texts:
            text.set_color('black')
        # Save plot
        fig.savefig(f"{outcome}.svg", format="svg")
        plt.show()

# With HP tuning 
def xgb_tune(X_train, y_train, X_test, y_test, outcome):

    # Calculate scale_pos_weight
    target_series = y_train[outcome].astype(int)
    scale_pos_weight = (target_series == 0).sum() / (target_series == 1).sum()

    # Hyperparameter tuning
    param_grid = {
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 1, 5],
        'min_child_weight': [1, 5, 10],
        'scale_pos_weight': [scale_pos_weight * 0.5, scale_pos_weight, scale_pos_weight * 2],
        'reg_alpha': [0,  0.1, 1, 10],     # L1 regularization
        'reg_lambda': [0,  0.1, 1, 10]     # L2 regularization
    }
    
    clf = XGBClassifier(
        objective='binary:logistic',
        eval_metric='aucpr',
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    randomized_search = RandomizedSearchCV(
        estimator=clf,
        param_distributions=param_grid,
        n_iter=50, 
        scoring='average_precision',
        cv=cv,
        n_jobs=1 
    )

    # Hyperparameter Fit
    randomized_search.fit(X_train, y_train[outcome])
    best_params = randomized_search.best_params_

    # Train final model
    final_model = XGBClassifier(
        **best_params,
        objective='binary:logistic',
        eval_metric='aucpr',
        random_state=42
    )

    # Fit final model
    final_model.fit(X_train, y_train[outcome])
    calibrator = CalibratedClassifierCV(final_model, method='isotonic', cv=5)

    # Step 2: Fit on training data
    calibrator.fit(X_train, y_train[outcome].astype(int))

    # Step 3: Predict calibrated probabilities
    y_pred = calibrator.predict(X_test)
    y_proba = calibrator.predict_proba(X_test)[:, 1]

    # Calculate AUC-PR
    pr_auc = average_precision_score(y_test, y_proba)
    print(f'PR AUC: {pr_auc:.2f}')
    # Calculate AUC
    auc = roc_auc_score(y_test, y_proba)
    print(f'AUC: {auc:.2f}')
    # Print report
    print(classification_report(y_test, y_pred, digits=2))

    # 4. Compute Spiegelhalter Z-statistic
    E = np.sum(y_proba)  
    O = np.sum(y_test.astype(int).values)  
    V = np.sum(y_proba * (1 - y_proba))  
    Z = (O - E) / np.sqrt(V)
    print(f"Spiegelhalter Z-statistic: {Z:.3f}")



In [148]:
# ElasticNet
def elasticnet(X_train, y_train, X_test, y_test, outcome):
    original_names = X_train.columns.tolist()

    # scale numeric columns
    num_cols = X_train.select_dtypes(include='number').columns.tolist()
    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_test_scaled[num_cols] = scaler.transform(X_test[num_cols])

    # define and fit the model
    model = LogisticRegressionCV(
        cv = 5,
        penalty='elasticnet',
        l1_ratios=[0.1,0.5,0.9],
        solver='saga',
        max_iter=10000,
        random_state=42
    )
    model.fit(X_train_scaled, y_train[outcome])

    # predict
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    # evaluate
    pr_auc = average_precision_score(y_test, y_proba)
    print(f'PR AUC: {pr_auc:.2f}')
    auc = roc_auc_score(y_test, y_proba)
    print(f'AUC: {auc:.2f}')
    print(classification_report(y_test, y_pred > 0.5, digits=2))

In [149]:
# TabNet
def tabnet(X_train, y_train, X_test, y_test, outcome):

    # make into np arrays
    X_train_np = X_train.to_numpy(dtype=np.float32)
    X_test_np = X_test.to_numpy(dtype=np.float32)
    y_train_np = y_train.to_numpy()
    y_train_np = y_train_np.ravel()
    y_test_np = y_test.to_numpy()
    y_test_np = y_test_np.ravel()

    # define the model
    model = TabNetClassifier(
        n_d=32,
        n_a=32,
        n_steps=3,
        gamma=1.5,
        n_independent=2,
        n_shared=2,
        momentum=0.3,
        lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.001),
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax',
        seed=42,
        verbose=0
    )

    # fit the model
    model.fit(
        X_train=X_train_np, y_train=y_train_np,
        eval_set=[(X_test_np, y_test_np)], 
        eval_name=['val'],
        eval_metric=['auc'], 
        max_epochs=200,
        patience=20,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )

    # predict
    y_pred = model.predict(X_test_np)

    # probability for class 1
    y_proba = model.predict_proba(X_test_np)[:, 1]

    # calculate AUC-PR
    pr_auc = average_precision_score(y_test, y_proba)
    print(f'PR AUC: {pr_auc:.2f}')
    auc = roc_auc_score(y_test, y_proba)
    print(f'AUC: {auc:.2f}')

    # print report
    print(classification_report(y_test, y_pred > 0.5, digits=2))

    return {
        'model': model
    }


In [150]:
# Predictors list
predictors_in = [
    'gender', 
    'age',
    'race',
    'arrival_transport',
    'pain',
    'temperature',
    'heartrate',    
    'resprate',
    'o2sat',
    'sbp',
    'dbp',
    'revisit_72hrs',
    'med_gsn_count',
    'acuity', 
    'cluster',
    'CCI_cat',
    'previous_cat', 
    'previous_result',
    'time_of_day'
    ]

In [151]:
# SHAP variables list
rename_map = {
    'CCI_cat_0.0':'CCI:0*',
    'CCI_cat_1.0':'CCI:1*',
    'CCI_cat_2.0':'CCI:2*',
    'CCI_cat_3.0':'CCI:3*',
    'age': 'Age(years)',
    'gender_M': 'Gender:Male*',
    'gender_F': 'Gender:Female*',
    'pain': 'Pain Score(/10)',
    'race_WHITE': 'Race:White*',
    'arrival_transport_ambulance': 'Arrival:Ambulance*',
    'arrival_transport_walk in': 'Arrival:Walk In*',
    'arrival_transport_unknown': 'Arrival:Unknown*',
    'cluster_1': 'Complaint:Influenza like illness*',
    'cluster_2': 'Complaint:Mental Status +\nVehicle Collision*',
    'cluster_3': 'Complaint:Chest pain +\nDyspnea*',
    'cluster_5': 'Complaint:Abdominal Pain*',
    'cluster_7': 'Complaint:Wound eval\ \nAbnormal labs*',
    'cluster_10': 'Complaint:Suicidal ideation*',
    'cluster_11': 'Complaint:Fall*',
    'dbp': 'Diastolic BP(mmhg)',
    'sbp': 'Systolic BP(mmhg)',
    'o2sat': 'Oxygen Saturation(%)',
    'heartrate': 'Heart Rate(bpm)',
    'resprate': 'Respiratory Rate(bpm)',
    'temperature': 'Temperature(F)',
    'med_gsn_count': 'Number of Medications(N)',
    'acuity': 'Acuity(/5)',
    }

## Home

### Data

#### Impute

In [ ]:
target_columns = ['home']

# XGBoost and Elastic net need different one hot encoding strategies
X_train_adxg, X_test_adxg, y_train_adxg, y_test_adxg = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

X_train_ad, X_test_ad, y_train_ad, y_test_ad = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=True,
    tab = 'not_tab'
)

X_train_adtab, X_test_adtab, y_train_adtab, y_test_adtab = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'tab'
)

#### Raw

In [ ]:
X_train_adraw, X_test_adraw, y_train_adraw, y_test_adraw = data_process(
    data, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

### XGBoost

#### Imputed data

In [ ]:
ad_xgb = xgb_f(X_train_adxg,  y_train_adxg,X_test_adxg, y_test_adxg, rename_map, 'home', shap_in=True)

with open('ad_xg.pkl', 'wb') as f:
    pickle.dump(ad_xgb, f)

del ad_xgb
gc.collect()

##### With tuning

In [ ]:
ad_xgb_tune = xgb_tune(X_train_adxg,  y_train_adxg,X_test_adxg, y_test_adxg,'home')

with open('ad_xg_tune.pkl', 'wb') as f:
    pickle.dump(ad_xgb_tune, f)

del ad_xgb_tune
gc.collect()

##### With no acuity data

In [ ]:
xgb_f(
    X_train_adxg.drop('acuity', axis = 1),  
    y_train_adxg,
    X_test_adxg.drop('acuity', axis = 1), 
    y_test_adxg, rename_map, 
    'home', 
    shap_in = False)

#### Raw Data

In [ ]:
ad_raw = xgb_f(X_train_adraw,  y_train_adraw,X_test_adraw, y_test_adraw, rename_map, 'home', shap_in = False)

with open('ad_raw.pkl', 'wb') as f:
    pickle.dump(ad_raw, f)

del ad_raw
gc.collect()

### Elastic net

In [ ]:
ad_el = elasticnet(X_train_ad, y_train_ad, X_test_ad, y_test_ad, 'home')

with open('ad_el.pkl', 'wb') as f:
    pickle.dump(ad_el, f)

del ad_el
gc.collect()

### Tabnet

In [ ]:
ad_tabnet = tabnet(X_train_adtab, y_train_adtab, X_test_adtab, y_test_adtab, 'admitted')

with open('ad_tabnet.pkl', 'wb') as f:
    pickle.dump(ad_tabnet, f)

del ad_tabnet
gc.collect()

In [160]:
admitted_data = {
    'X_train_adxg': X_train_adxg,
    'X_test_adxg': X_test_adxg,
    'y_train_adxg': y_train_adxg,
    'y_test_adxg': y_test_adxg,
    'X_train_ad': X_train_ad,
    'X_test_ad': X_test_ad,
    'y_train_ad': y_train_ad,
    'y_test_ad': y_test_ad,
    'X_train_adtab': X_train_adtab,
    'X_test_adtab': X_test_adtab,
    'y_train_adtab': y_train_adtab,
    'y_test_adtab': y_test_adtab,
    'X_train_adraw': X_train_adraw,
    'X_test_adraw': X_test_adraw,
    'y_train_adraw': y_train_adraw,
    'y_test_adraw': y_test_adraw
}

with open('admitted_data.pkl', 'wb') as f:
    pickle.dump(admitted_data, f)
    
del X_train_adxg, X_test_adxg, y_train_adxg, y_test_adxg
del X_train_ad, X_test_ad, y_train_ad, y_test_ad
del X_train_adtab, X_test_adtab, y_train_adtab, y_test_adtab

## Diagnosis

### Data

#### Impute

In [ ]:
target_columns = ['nyu']

# XGBoost and Elastic net need different one hot encoding strategies
X_train_diagxg, X_test_diagxg, y_train_diagxg, y_test_diagxg = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

X_train_diag, X_test_diag, y_train_diag, y_test_diag = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=True,
    tab = 'not_tab'
)

X_train_diagtab, X_test_diagtab, y_train_diagtab, y_test_diagtab = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'tab'
)

X_train_diagac, X_test_diagac, y_train_diagac, y_test_diagac = data_process(
    data_impute, 
    predictors_ac, 
    target_columns, 
    drop_first_in=True,
    tab = 'tab'
)

In [ ]:
target_columns = ['nyu']
X_train_diagac, X_test_diagac, y_train_diagac, y_test_diagac = data_process(
    data_impute, 
    predictors_ac, 
    target_columns, 
    drop_first_in=True,
    tab = 'tab'
)

#### Raw

In [ ]:
X_train_diagraw, X_test_diagraw, y_train_diagraw, y_test_diagraw = data_process(
    data, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

### XGBoost

#### Imputed data

In [ ]:
nyu_xgb = xgb_f(X_train_diagxg,  y_train_diagxg,X_test_diagxg, y_test_diagxg, rename_map, 'nyu', shap_in = True)

with open('nyu_xgb.pkl', 'wb') as f:
    pickle.dump(nyu_xgb, f)

##### With tuning

In [ ]:
diag_xgb_tune = xgb_tune(X_train_diagxg,  y_train_diagxg,X_test_diagxg, y_test_diagxg,'nyu')

with open('diag_xgb_tune.pkl', 'wb') as f:
    pickle.dump(diag_xgb_tune, f)

del diag_xgb_tune
gc.collect()

##### With no acuity score

In [ ]:
diag_xgb_acuity = xgb_f(
    X_train_diagxg.drop('acuity', axis = 1),  
    y_train_diagxg,
    X_test_diagxg.drop('acuity', axis = 1), 
    y_test_diagxg, 
    rename_map,
    'nyu',
    shap_in = False)

#### Raw Data

In [ ]:
diag_raw = xgb_f(X_train_diagraw, y_train_diagraw, X_test_diagraw, y_test_diagraw, rename_map, 'nyu', shap_in = False)

with open('diag_raw.pkl', 'wb') as f:
    pickle.dump(diag_raw, f)

del diag_raw
gc.collect()

### Elastic net

In [ ]:
nyu_el = elasticnet(X_train_diag, y_train_diag, X_test_diag, y_test_diag, 'nyu')

with open('nyu_el.pkl', 'wb') as f:
    pickle.dump(nyu_el, f)

del nyu_el
gc.collect()

### Tabnet

In [ ]:
nyu_tabnet = tabnet(X_train_diagtab, y_train_diagtab, X_test_diagtab, y_test_diagtab, 'nyu')

with open('nyu_tabnet.pkl', 'wb') as f:
    pickle.dump(nyu_tabnet, f)

del nyu_tabnet
gc.collect()

In [ ]:
nyu_data = {
    'X_train_diagxg': X_train_diagxg,
    'X_test_diagxg': X_test_diagxg,
    'y_train_diagxg': y_train_diagxg,
    'y_test_diagxg': y_test_diagxg,
    'X_train_diag': X_train_diag,
    'X_test_diag': X_test_diag,
    'y_train_diag': y_train_diag,
    'y_test_diag': y_test_diag,
    'X_train_diagtab': X_train_diagtab,
    'X_test_diagtab': X_test_diagtab,
    'y_train_diagtab': y_train_diagtab,
    'y_test_diagtab': y_test_diagtab,
    'X_train_diagraw': X_train_diagraw,
    'X_test_diagraw': X_test_diagraw,
    'y_train_diagraw': y_train_diagraw,
    'y_test_diagraw': y_test_diagraw
}

with open('nyu_data.pkl', 'wb') as f:
    pickle.dump(nyu_data, f)
    
del X_train_diagxg, X_test_diagxg, y_train_diagxg, y_test_diagxg
del X_train_diag, X_test_diag, y_train_diag, y_test_diag
del X_train_diagtab, X_test_diagtab, y_train_diagtab, y_test_diagtab
gc.collect()

## Resources

### Data 

#### Impute

In [ ]:
target_columns = ['res']

# XGBoost and Elastic net need different one hot encoding strategies
X_train_resxg, X_test_resxg, y_train_resxg, y_test_resxg = data_process(
    data_impute,
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

X_train_res, X_test_res, y_train_res, y_test_res = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=True,
    tab = 'not_tab'
)

X_train_restab, X_test_restab, y_train_restab, y_test_restab = data_process(
    data_impute, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'tab'
)

#### Raw

In [ ]:
X_train_resraw, X_test_resraw, y_train_resraw, y_test_resraw = data_process(
    data, 
    predictors_in, 
    target_columns, 
    drop_first_in=False,
    tab = 'not_tab'
)

### XGBoost

#### Imputed data

In [ ]:
res_xgb = xgb_f(X_train_resxg, y_train_resxg, X_test_resxg, y_test_resxg, rename_map, 'res', shap_in=True)

with open('res_xg.pkl', 'wb') as f:
    pickle.dump(res_xgb, f)

##### With tuning

In [ ]:
res_tune = xgb_tune(X_train_resxg, y_train_resxg, X_test_resxg, y_test_resxg, 'res')

with open('res_tune.pkl', 'wb') as f:
    pickle.dump(res_tune, f)

del res_tune
gc.collect()

##### With no acuity data

In [ ]:
xgb_f(
    X_train_resxg.drop('acuity', axis = 1), 
    y_train_resxg, 
    X_test_resxg.drop('acuity', axis = 1), 
    y_test_resxg, 
    rename_map, 
    'res',
    shap_in = False
)

#### Raw data

In [ ]:
res_raw = xgb_f(X_train_resraw, y_train_resraw, X_test_resraw, y_test_resraw, rename_map, 'res', shap_in = False)

with open('res_raw.pkl', 'wb') as f:
    pickle.dump(res_raw, f)

del res_raw
gc.collect()

### Elastic net

In [ ]:
res_el = elasticnet(X_train_res, y_train_res, X_test_res, y_test_res, 'res')

with open('res_el.pkl', 'wb') as f:
    pickle.dump(res_el, f)

del res_el
gc.collect()

### Tabnet

In [ ]:
res_tabnet = tabnet(X_train_res, y_train_res, X_test_res, y_test_res, 'res')

with open('res_tabnet.pkl', 'wb') as f:
    pickle.dump(res_tabnet, f)

del res_tabnet
gc.collect()

In [ ]:
res_data = {
    'X_train_resxg': X_train_resxg,
    'X_test_resxg': X_test_resxg,
    'y_train_resxg': y_train_resxg,
    'y_test_resxg': y_test_resxg,
    'X_train_res': X_train_res,
    'X_test_res': X_test_res,
    'y_train_res': y_train_res,
    'y_test_res': y_test_res,
    'X_train_restab': X_train_restab,
    'X_test_restab': X_test_restab,
    'y_train_restab': y_train_restab,
    'y_test_restab': y_test_restab,
    'X_train_resraw': X_train_resraw,
    'X_test_resraw': X_test_resraw,
    'y_train_resraw': y_train_resraw,
    'y_test_resraw': y_test_resraw
}


with open('res_data.pkl', 'wb') as f:
    pickle.dump(res_data, f)
    
del X_train_resxg, X_test_resxg, y_train_resxg, y_test_resxg
del X_train_res, X_test_res, y_train_res, y_test_res
del X_train_restab, X_test_restab, y_train_restab, y_test_restab
gc.collect()